# DeepFabric ile Sentetik Veri Üretimi - OpenRouter + GPT-OSS-20B

Bu notebook, DeepFabric kütüphanesini kullanarak **OpenRouter API** üzerinden **GPT-OSS-20B** modeli ile sentetik veri üretimini göstermektedir.

## İçindekiler
1. Kurulum ve Bağımlılıklar
2. OpenRouter API Yapılandırması
3. Farklı Türde Sentetik Veri Üretimi:
   - Chain of Thought (Adım Adım Akıl Yürütme)
   - Tool Calling (Fonksiyon Çağırma)
   - Multi-Turn Conversations (Çok Turlu Konuşmalar)
   - Question-Answer Pairs (Soru-Cevap Çiftleri)
   - Code Generation (Kod Üretimi)
4. Veri Analizi ve Görselleştirme
5. Fine-tuning için Hazırlık

---

## 1. Kurulum ve Bağımlılıklar

### DeepFabric ve Gerekli Kütüphaneleri Kurma

In [ ]:
# DeepFabric kütüphanesini GitHub'dan kuruyoruz
!pip install git+https://github.com/lukehinds/deepfabric.git -q

# Gerekli yardımcı kütüphaneleri kuruyoruz
!pip install datasets pandas matplotlib seaborn pyyaml -q

print("✅ Tüm kütüphaneler başarıyla kuruldu!")

### Kütüphaneleri İçe Aktarma

In [ ]:
import json
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datasets import load_dataset
import subprocess
import os
from getpass import getpass

# Görselleştirme ayarları
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ Kütüphaneler başarıyla yüklendi!")

## 2. OpenRouter API Yapılandırması

### API Anahtarını Ayarlama

OpenRouter API kullanmak için bir API anahtarına ihtiyacınız var:
1. [OpenRouter](https://openrouter.ai/) sitesine gidin
2. Hesap oluşturun veya giriş yapın
3. API anahtarınızı alın
4. Aşağıdaki hücrede API anahtarınızı girin

In [ ]:
# OpenRouter API anahtarını ayarla
OPENROUTER_API_KEY = getpass("OpenRouter API Anahtarınızı girin: ")

# Environment variable olarak ayarla
os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY

print("✅ API anahtarı ayarlandı!")

### Çalışma Dizinlerini Oluşturma

In [ ]:
# Yapılandırma dizini oluştur
config_dir = Path('deepfabric_configs')
config_dir.mkdir(exist_ok=True)

# Çıktı dizini oluştur
output_dir = Path('synthetic_datasets')
output_dir.mkdir(exist_ok=True)

print(f"✅ Dizinler oluşturuldu!")
print(f"   📁 Yapılandırma dizini: {config_dir}")
print(f"   📁 Çıktı dizini: {output_dir}")

## 3. Sentetik Veri Üretimi

---

## 3.1 Chain of Thought (Adım Adım Akıl Yürütme)

Bu tip veri, modellere adım adım düşünmeyi ve problem çözmeyi öğretmek için kullanılır.

In [ ]:
# Chain of Thought yapılandırması - OpenRouter + GPT-OSS-20B
cot_config = {
    'dataset_system_prompt': 'Sen bir matematik ve programlama uzmanısın. Öğrencilere adım adım problem çözmeyi öğretiyorsun. Her problem için düşünce sürecini detaylı açıklıyorsun.',
    
    'topic_graph': {
        'topic_prompt': 'Matematik ve Programlama Problemleri: Algoritmalar, veri yapıları, ve matematiksel çözümlemeler',
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.7,
        'degree': 3,  # Her düğümden 3 alt konu üretilecek
        'depth': 2,   # 2 seviye derinlik
        'save_as': str(output_dir / 'topics_cot.jsonl')
    },
    
    'data_engine': {
        'generation_system_prompt': 'Her problemi adım adım çöz ve düşünce sürecini açıkça göster.',
        'instructions': 'Adım adım akıl yürütme ile matematik ve programlama problemleri çöz.',
        'conversation_type': 'chain_of_thought',
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.5,
        'max_retries': 3
    },
    
    'dataset': {
        'save_as': str(output_dir / 'cot_math_programming.jsonl'),
        'creation': {
            'num_steps': 15,  # 15 örnek üret
            'batch_size': 3,
            'sys_msg': True
        }
    }
}

# YAML dosyası olarak kaydet
cot_config_path = config_dir / 'cot_openrouter_config.yaml'
with open(cot_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(cot_config, f, allow_unicode=True)

print(f"✅ Chain of Thought yapılandırması kaydedildi: {cot_config_path}")
print("\n📋 Yapılandırma özeti:")
print(f"  • Konu derinliği: {cot_config['topic_graph']['depth']}")
print(f"  • Konu genişliği: {cot_config['topic_graph']['degree']}")
print(f"  • Örnek sayısı: {cot_config['dataset']['creation']['num_steps']}")
print(f"  • Batch boyutu: {cot_config['dataset']['creation']['batch_size']}")

In [ ]:
# Chain of Thought verisi üret
cmd = [
    'deepfabric', 'generate',
    str(cot_config_path)
]

print("🚀 Chain of Thought verisi üretiliyor...")
print(f"📝 Komut: {' '.join(cmd)}")
print(f"📁 Config dosyası: {cot_config_path}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("⚠️ Uyarılar:")
    print(result.stderr)

# Çıktı dosyalarını kontrol et
output_file_cot = output_dir / 'cot_math_programming.jsonl'
if output_file_cot.exists():
    print("\n" + "="*80)
    print(f"✅ Chain of Thought verisi oluşturuldu: {output_file_cot}")
    print(f"📦 Dosya boyutu: {output_file_cot.stat().st_size / 1024:.2f} KB")
else:
    print("\n" + "="*80)
    print(f"⚠️ Çıktı dosyası bulunamadı: {output_file_cot}")

---

## 3.2 Tool Calling (Fonksiyon Çağırma)

Bu tip veri, modellere API fonksiyonlarını doğru seçip çağırmayı öğretir.

In [ ]:
# Tool Calling yapılandırması
tool_config = {
    'dataset_system_prompt': 'Sen bir API yardımcısısın. Kullanıcılara doğru fonksiyonları seçip çağırmayı, parametreleri doğru belirlemeyi öğretiyorsun.',
    
    'topic_graph': {
        'topic_prompt': 'API ve Fonksiyon Çağırma: Web servisleri, veritabanı işlemleri, dosya yönetimi ve veri dönüşümleri',
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.8,
        'degree': 4,
        'depth': 2,
        'save_as': str(output_dir / 'topics_tool_calling.jsonl')
    },
    
    'data_engine': {
        'generation_system_prompt': 'Her işlem için hangi tool kullanılacağını açıkla ve doğru parametreleri belirle.',
        'instructions': 'API fonksiyon çağrılarını doğru bir şekilde göster.',
        'conversation_type': 'basic',  # Tool calling için basic type kullanıyoruz
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.3,
        'max_retries': 3
    },
    
    'dataset': {
        'save_as': str(output_dir / 'tool_calling_api.jsonl'),
        'creation': {
            'num_steps': 12,
            'batch_size': 3,
            'sys_msg': True
        }
    }
}

tool_config_path = config_dir / 'tool_calling_config.yaml'
with open(tool_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(tool_config, f, allow_unicode=True)

print(f"✅ Tool Calling yapılandırması kaydedildi: {tool_config_path}")

In [ ]:
# Tool Calling verisi üret
cmd = [
    'deepfabric', 'generate',
    str(tool_config_path)
]

print("🚀 Tool Calling verisi üretiliyor...")
print(f"📁 Config dosyası: {tool_config_path}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("⚠️ Uyarılar:")
    print(result.stderr)

output_file_tool = output_dir / 'tool_calling_api.jsonl'
if output_file_tool.exists():
    print("\n" + "="*80)
    print(f"✅ Tool Calling verisi oluşturuldu: {output_file_tool}")
    print(f"📦 Dosya boyutu: {output_file_tool.stat().st_size / 1024:.2f} KB")
else:
    print("\n" + "="*80)
    print(f"⚠️ Çıktı dosyası bulunamadı: {output_file_tool}")

---

## 3.3 Multi-Turn Conversations (Çok Turlu Konuşmalar)

Bu tip veri, modellere uzun süreli konuşma yeteneği kazandırır.

In [ ]:
# Multi-turn conversation yapılandırması
multiturn_config = {
    'dataset_system_prompt': 'Sen yardımcı bir teknoloji danışmanısın. Kullanıcılarla doğal, çok turlu konuşmalar yaparak sorunlarını adım adım çözüyorsun.',
    
    'topic_graph': {
        'topic_prompt': 'Yazılım Geliştirme Destek: Debugging, kod optimizasyonu, mimari kararlar ve best practices',
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.7,
        'degree': 3,
        'depth': 2,
        'save_as': str(output_dir / 'topics_multiturn.jsonl')
    },
    
    'data_engine': {
        'generation_system_prompt': 'Her turda bağlamı hatırlayarak yardımcı ol ve problemi adım adım çöz.',
        'instructions': 'Kullanıcıyla çok turlu, doğal konuşmalar yap.',
        'conversation_type': 'basic',  # Multi-turn konuşmalar için
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.6,
        'max_retries': 3
    },
    
    'dataset': {
        'save_as': str(output_dir / 'multiturn_tech_support.jsonl'),
        'creation': {
            'num_steps': 10,
            'batch_size': 2,
            'sys_msg': True
        }
    }
}

multiturn_config_path = config_dir / 'multiturn_config.yaml'
with open(multiturn_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(multiturn_config, f, allow_unicode=True)

print(f"✅ Multi-turn yapılandırması kaydedildi: {multiturn_config_path}")

In [ ]:
# Multi-turn verisi üret
cmd = [
    'deepfabric', 'generate',
    str(multiturn_config_path)
]

print("🚀 Multi-turn conversation verisi üretiliyor...")
print(f"📁 Config dosyası: {multiturn_config_path}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("⚠️ Uyarılar:")
    print(result.stderr)

output_file_multiturn = output_dir / 'multiturn_tech_support.jsonl'
if output_file_multiturn.exists():
    print("\n" + "="*80)
    print(f"✅ Multi-turn verisi oluşturuldu: {output_file_multiturn}")
    print(f"📦 Dosya boyutu: {output_file_multiturn.stat().st_size / 1024:.2f} KB")
else:
    print("\n" + "="*80)
    print(f"⚠️ Çıktı dosyası bulunamadı: {output_file_multiturn}")

---

## 3.4 Question-Answer Pairs (Soru-Cevap Çiftleri)

Basit soru-cevap formatında eğitim verisi üretimi.

In [ ]:
# Q&A yapılandırması
qa_config = {
    'dataset_system_prompt': 'Sen bir eğitim asistanısın. Kullanıcı sorularına net, açıklayıcı ve doğru cevaplar veriyorsun.',
    
    'topic_graph': {
        'topic_prompt': 'Python ve Makine Öğrenmesi: Temel kavramlar, kütüphaneler, algoritmalar ve uygulamalar',
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.7,
        'degree': 4,
        'depth': 2,
        'save_as': str(output_dir / 'topics_qa.jsonl')
    },
    
    'data_engine': {
        'generation_system_prompt': 'Net ve açıklayıcı cevaplar ver.',
        'instructions': 'Python ve makine öğrenmesi konularında eğitim amaçlı Q&A çiftleri oluştur.',
        'conversation_type': 'basic',
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.4,
        'max_retries': 3
    },
    
    'dataset': {
        'save_as': str(output_dir / 'qa_python_ml.jsonl'),
        'creation': {
            'num_steps': 20,
            'batch_size': 4,
            'sys_msg': True
        }
    }
}

qa_config_path = config_dir / 'qa_config.yaml'
with open(qa_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(qa_config, f, allow_unicode=True)

print(f"✅ Q&A yapılandırması kaydedildi: {qa_config_path}")

In [ ]:
# Q&A verisi üret
cmd = [
    'deepfabric', 'generate',
    str(qa_config_path)
]

print("🚀 Q&A verisi üretiliyor...")
print(f"📁 Config dosyası: {qa_config_path}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("⚠️ Uyarılar:")
    print(result.stderr)

output_file_qa = output_dir / 'qa_python_ml.jsonl'
if output_file_qa.exists():
    print("\n" + "="*80)
    print(f"✅ Q&A verisi oluşturuldu: {output_file_qa}")
    print(f"📦 Dosya boyutu: {output_file_qa.stat().st_size / 1024:.2f} KB")
else:
    print("\n" + "="*80)
    print(f"⚠️ Çıktı dosyası bulunamadı: {output_file_qa}")

---

## 3.5 Code Generation (Kod Üretimi)

Kod yazma ve açıklama için özelleştirilmiş veri üretimi.

In [ ]:
# Code Generation yapılandırması
code_config = {
    'dataset_system_prompt': 'Sen uzman bir yazılım geliştiricisisin. Kullanıcıların isteklerine göre temiz, iyi dokümante edilmiş ve best practice kod örnekleri yazıyorsun.',
    
    'topic_graph': {
        'topic_prompt': 'Python Kod Örnekleri: Web scraping, veri analizi, API geliştirme, automation ve utilities',
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.6,
        'degree': 3,
        'depth': 2,
        'save_as': str(output_dir / 'topics_code.jsonl')
    },
    
    'data_engine': {
        'generation_system_prompt': 'Her kod bloğunu detaylı açıkla ve best practice uygula.',
        'instructions': 'Python kod örnekleri ile açıklamalar yaz.',
        'conversation_type': 'basic',
        'provider': 'openrouter',
        'model': 'neuralmagic/llama-3.1-nemotron-70b-instruct-hf',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.3,
        'max_retries': 3
    },
    
    'dataset': {
        'save_as': str(output_dir / 'code_generation_python.jsonl'),
        'creation': {
            'num_steps': 15,
            'batch_size': 3,
            'sys_msg': True
        }
    }
}

code_config_path = config_dir / 'code_generation_config.yaml'
with open(code_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(code_config, f, allow_unicode=True)

print(f"✅ Code Generation yapılandırması kaydedildi: {code_config_path}")

In [ ]:
# Code Generation verisi üret
cmd = [
    'deepfabric', 'generate',
    str(code_config_path)
]

print("🚀 Code Generation verisi üretiliyor...")
print(f"📁 Config dosyası: {code_config_path}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("⚠️ Uyarılar:")
    print(result.stderr)

output_file_code = output_dir / 'code_generation_python.jsonl'
if output_file_code.exists():
    print("\n" + "="*80)
    print(f"✅ Code Generation verisi oluşturuldu: {output_file_code}")
    print(f"📦 Dosya boyutu: {output_file_code.stat().st_size / 1024:.2f} KB")
else:
    print("\n" + "="*80)
    print(f"⚠️ Çıktı dosyası bulunamadı: {output_file_code}")

---

## 4. Veri Analizi ve Görselleştirme

Üretilen tüm veri setlerini analiz edelim.

In [ ]:
def load_jsonl(file_path):
    """JSONL dosyasını yükle"""
    data = []
    if not Path(file_path).exists():
        print(f"⚠️ Dosya bulunamadı: {file_path}")
        return data
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():  # Boş satırları atla
                data.append(json.loads(line))
    return data

def analyze_dataset(data, dataset_name):
    """Veri setini detaylı analiz et"""
    print(f"\n{'='*80}")
    print(f"📊 Dataset: {dataset_name}")
    print(f"{'='*80}")
    print(f"Toplam örnek sayısı: {len(data)}")
    
    if len(data) > 0:
        # İlk örneği göster
        print("\n📝 İlk Örnek (ilk 500 karakter):")
        print("-" * 80)
        print(json.dumps(data[0], indent=2, ensure_ascii=False)[:500] + "...")
        
        # Mesaj istatistikleri
        message_lengths = []
        total_messages = 0
        
        for item in data:
            if 'messages' in item:
                total_messages += len(item['messages'])
                for msg in item['messages']:
                    if 'content' in msg:
                        message_lengths.append(len(msg['content']))
        
        if message_lengths:
            print(f"\n📈 İstatistikler:")
            print(f"  • Toplam mesaj sayısı: {total_messages}")
            print(f"  • Ortalama konuşma uzunluğu: {total_messages/len(data):.1f} mesaj/örnek")
            print(f"  • Ortalama mesaj uzunluğu: {sum(message_lengths)/len(message_lengths):.0f} karakter")
            print(f"  • En kısa mesaj: {min(message_lengths)} karakter")
            print(f"  • En uzun mesaj: {max(message_lengths)} karakter")
    
    return data

print("✅ Analiz fonksiyonları hazır!")

In [ ]:
# Tüm veri setlerini yükle ve analiz et
datasets = {
    'Chain of Thought': output_file_cot,
    'Tool Calling': output_file_tool,
    'Multi-Turn': output_file_multiturn,
    'Q&A Pairs': output_file_qa,
    'Code Generation': output_file_code,
}

loaded_datasets = {}
for name, path in datasets.items():
    data = load_jsonl(path)
    loaded_datasets[name] = analyze_dataset(data, name)

In [ ]:
# Veri setlerini görselleştir
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Veri seti boyutları
dataset_sizes = {name: len(data) for name, data in loaded_datasets.items() if len(data) > 0}
if dataset_sizes:
    colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6']
    bars = axes[0, 0].bar(range(len(dataset_sizes)), list(dataset_sizes.values()), color=colors[:len(dataset_sizes)])
    axes[0, 0].set_xticks(range(len(dataset_sizes)))
    axes[0, 0].set_xticklabels(list(dataset_sizes.keys()), rotation=45, ha='right')
    axes[0, 0].set_xlabel('Veri Seti', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylabel('Örnek Sayısı', fontsize=12, fontweight='bold')
    axes[0, 0].set_title('📊 Veri Seti Boyutları', fontsize=14, fontweight='bold')
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # Değerleri barların üzerine ekle
    for bar in bars:
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                       f'{int(height)}',
                       ha='center', va='bottom', fontweight='bold')

# 2. Mesaj uzunluk dağılımı (box plot)
all_lengths = []
labels = []
for name, data in loaded_datasets.items():
    if len(data) > 0:
        lengths = []
        for item in data:
            if 'messages' in item:
                for msg in item['messages']:
                    if 'content' in msg:
                        lengths.append(len(msg['content']))
        if lengths:
            all_lengths.append(lengths)
            labels.append(name)

if all_lengths:
    bp = axes[0, 1].boxplot(all_lengths, labels=labels, patch_artist=True)
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    axes[0, 1].set_xlabel('Veri Seti', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Mesaj Uzunluğu (karakter)', fontsize=12, fontweight='bold')
    axes[0, 1].set_title('📏 Mesaj Uzunluk Dağılımı', fontsize=14, fontweight='bold')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Toplam karakter sayısı
total_chars = {}
for name, data in loaded_datasets.items():
    if len(data) > 0:
        char_count = 0
        for item in data:
            if 'messages' in item:
                for msg in item['messages']:
                    if 'content' in msg:
                        char_count += len(msg['content'])
        total_chars[name] = char_count / 1000  # KB cinsinden

if total_chars:
    bars = axes[1, 0].bar(range(len(total_chars)), list(total_chars.values()), color=colors[:len(total_chars)])
    axes[1, 0].set_xticks(range(len(total_chars)))
    axes[1, 0].set_xticklabels(list(total_chars.keys()), rotation=45, ha='right')
    axes[1, 0].set_xlabel('Veri Seti', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Toplam Boyut (K karakter)', fontsize=12, fontweight='bold')
    axes[1, 0].set_title('💾 Toplam Veri Boyutu', fontsize=14, fontweight='bold')
    axes[1, 0].grid(axis='y', alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.0f}K',
                       ha='center', va='bottom', fontweight='bold')

# 4. Ortalama konuşma uzunluğu (mesaj sayısı)
avg_conv_length = {}
for name, data in loaded_datasets.items():
    if len(data) > 0:
        total_msgs = 0
        for item in data:
            if 'messages' in item:
                total_msgs += len(item['messages'])
        avg_conv_length[name] = total_msgs / len(data)

if avg_conv_length:
    bars = axes[1, 1].bar(range(len(avg_conv_length)), list(avg_conv_length.values()), color=colors[:len(avg_conv_length)])
    axes[1, 1].set_xticks(range(len(avg_conv_length)))
    axes[1, 1].set_xticklabels(list(avg_conv_length.keys()), rotation=45, ha='right')
    axes[1, 1].set_xlabel('Veri Seti', fontsize=12, fontweight='bold')
    axes[1, 1].set_ylabel('Ortalama Mesaj Sayısı', fontsize=12, fontweight='bold')
    axes[1, 1].set_title('💬 Ortalama Konuşma Uzunluğu', fontsize=14, fontweight='bold')
    axes[1, 1].grid(axis='y', alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.1f}',
                       ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / 'dataset_analysis_comprehensive.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ Görselleştirme kaydedildi: {output_dir / 'dataset_analysis_comprehensive.png'}")

### Örnek Konuşmaları İnceleme

Her veri setinden birer örnek görelim.

In [ ]:
def display_example(data, dataset_name, index=0):
    """Belirli bir örneği güzel formatta göster"""
    if len(data) == 0:
        print(f"⚠️ {dataset_name} veri seti boş!")
        return
    
    if index >= len(data):
        index = 0
    
    print(f"\n{'='*80}")
    print(f"📖 Dataset: {dataset_name} - Örnek #{index + 1}/{len(data)}")
    print(f"{'='*80}\n")
    
    item = data[index]
    
    if 'messages' in item:
        for i, msg in enumerate(item['messages'], 1):
            role = msg.get('role', 'unknown').upper()
            content = msg.get('content', '')
            
            # Rol bazlı emoji
            emoji = {'SYSTEM': '⚙️', 'USER': '👤', 'ASSISTANT': '🤖'}.get(role, '💬')
            
            print(f"{emoji} [{role}] - Mesaj {i}/{len(item['messages'])}")
            print(f"{'-'*80}")
            print(content[:500] + ("..." if len(content) > 500 else ""))  # İlk 500 karakter
            print(f"\n")
    else:
        print(json.dumps(item, indent=2, ensure_ascii=False)[:1000])

# Her veri setinden birer örnek göster
for name, data in loaded_datasets.items():
    if len(data) > 0:
        display_example(data, name, index=0)

---

## 5. Veri Setlerini Birleştirme ve Fine-tuning Hazırlığı

In [ ]:
# Veri setlerini birleştir
combined_data = []
dataset_distribution = {}

for name, data in loaded_datasets.items():
    if len(data) > 0:
        combined_data.extend(data)
        dataset_distribution[name] = len(data)

# Birleştirilmiş veri setini kaydet
combined_output = output_dir / 'combined_openrouter_dataset.jsonl'
with open(combined_output, 'w', encoding='utf-8') as f:
    for item in combined_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✅ Birleştirilmiş veri seti kaydedildi: {combined_output}")
print(f"\n📊 Toplam İstatistikler:")
print(f"  • Toplam örnek sayısı: {len(combined_data)}")
print(f"\n📈 Veri Seti Dağılımı:")
for name, count in dataset_distribution.items():
    percentage = (count / len(combined_data)) * 100
    print(f"  • {name:20s}: {count:3d} örnek ({percentage:5.1f}%)")

# Dosya boyutu bilgisi
file_size = combined_output.stat().st_size / (1024 * 1024)  # MB
print(f"\n💾 Dosya boyutu: {file_size:.2f} MB")

In [ ]:
# HuggingFace datasets kütüphanesi ile yükle
try:
    dataset = load_dataset('json', data_files=str(combined_output))
    print("✅ Veri seti HuggingFace Datasets ile başarıyla yüklendi!")
    print(f"\n📋 Veri seti bilgileri:")
    print(dataset)
    
    # Örnek bir satır göster
    if len(dataset['train']) > 0:
        print("\n📝 HuggingFace formatında örnek bir satır:")
        print(json.dumps(dataset['train'][0], indent=2, ensure_ascii=False)[:500] + "...")
except Exception as e:
    print(f"⚠️ Hata: {e}")

---

## 6. Fine-tuning Örnek Kodu

Üretilen veri ile model fine-tuning için örnek kod.

In [ ]:
# Fine-tuning için örnek kod
finetuning_code = '''
# ============================================================================
# DeepFabric ile Üretilen Veri ile Model Fine-tuning
# ============================================================================

# Gerekli kütüphaneleri kur:
# !pip install transformers trl peft accelerate bitsandbytes

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
import torch

# ============================================================================
# 1. Model ve Tokenizer Yükleme
# ============================================================================

model_name = "meta-llama/Llama-3.2-3B-Instruct"  # veya istediğiniz model

# 4-bit quantization için yapılandırma (GPU belleği tasarrufu)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Model yükle
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Tokenizer yükle
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Modeli QLoRA için hazırla
model = prepare_model_for_kbit_training(model)

# ============================================================================
# 2. Veri Setini Yükle
# ============================================================================

dataset = load_dataset(
    "json",
    data_files="synthetic_datasets/combined_openrouter_dataset.jsonl"
)

# Veri setini train/test olarak böl (opsiyonel)
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

print(f"Train set: {len(dataset['train'])} örnekler")
print(f"Test set: {len(dataset['test'])} örnekler")

# ============================================================================
# 3. LoRA Yapılandırması
# ============================================================================

peft_config = LoraConfig(
    r=16,                    # LoRA rank
    lora_alpha=32,          # LoRA alpha
    lora_dropout=0.05,      # Dropout
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[         # Hangi modüllere LoRA uygulanacak
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

# ============================================================================
# 4. Training Argümanları
# ============================================================================

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    evaluation_strategy="steps",
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="none",  # "wandb" için WandB logging
)

# ============================================================================
# 5. SFTTrainer Oluştur ve Eğit
# ============================================================================

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=2048,
    dataset_text_field="messages",  # DeepFabric formatı için
    packing=False,
)

# Eğitimi başlat
print("🚀 Fine-tuning başlıyor...")
trainer.train()

# ============================================================================
# 6. Modeli Kaydet
# ============================================================================

# LoRA adapterlerini kaydet
trainer.model.save_pretrained("./finetuned_model_lora")
tokenizer.save_pretrained("./finetuned_model_lora")

print("✅ Model kaydedildi: ./finetuned_model_lora")

# ============================================================================
# 7. Test ve Inference
# ============================================================================

# Modeli test et
def test_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Örnek test
test_prompt = "Bir binary search tree'yi Python ile nasıl implement ederim?"
print(f"\nTest Sorusu: {test_prompt}")
print(f"\nModel Cevabı:\n{test_model(test_prompt)}")
'''

# Kodu dosyaya kaydet
finetuning_file = output_dir / 'example_finetuning_openrouter.py'
with open(finetuning_file, 'w', encoding='utf-8') as f:
    f.write(finetuning_code)

print("📝 Fine-tuning Örnek Kodu")
print("="*80)
print(finetuning_code[:1000] + "\n... (devam ediyor) ...")
print("\n" + "="*80)
print(f"✅ Tam kod kaydedildi: {finetuning_file}")

---

## 7. Özet ve Sonraki Adımlar

### ✅ Tamamlananlar:

1. **DeepFabric Kurulumu**: Kütüphane ve bağımlılıklar kuruldu
2. **OpenRouter Entegrasyonu**: GPT-OSS-20B modeli ile bağlantı sağlandı
3. **5 Farklı Sentetik Veri Üretimi**:
   - ⚡ Chain of Thought (Adım adım akıl yürütme)
   - 🛠️ Tool Calling (Fonksiyon çağırma)
   - 💬 Multi-Turn Conversations (Çok turlu konuşmalar)
   - ❓ Question-Answer Pairs (Soru-cevap çiftleri)
   - 💻 Code Generation (Kod üretimi)
4. **Veri Analizi**: Detaylı istatistikler ve görselleştirmeler
5. **Fine-tuning Hazırlığı**: HuggingFace uyumlu format ve örnek kod

---

### 🚀 Sonraki Adımlar:

#### 1. Daha Fazla Veri Üretimi
- `num_steps` parametresini artırarak daha fazla örnek üretin
- Farklı `topic_prompt`'lar ile çeşitli konularda veri üretin
- `temperature` değerlerini ayarlayarak çeşitliliği kontrol edin

#### 2. Veri Kalitesi İyileştirme
- System prompt'ları özelleştirerek format kontrolü sağlayın
- Farklı reasoning template'leri deneyin
- Manuel inceleme ile kalitesiz örnekleri filtreleyin

#### 3. Model Fine-tuning
- Yukarıdaki örnek kodu kullanarak model eğitin
- QLoRA ile hafıza verimliliği sağlayın
- Farklı hyperparameter'lar deneyin

#### 4. Değerlendirme
- Fine-tune edilmiş modeli test veri seti ile değerlendirin
- Benchmark'larda performans ölçümü yapın
- İyileştirmeler için feedback loop oluşturun

---

### 📚 Faydalı Kaynaklar:

- 📖 [DeepFabric Dokümantasyonu](https://github.com/lukehinds/deepfabric)
- 🌐 [OpenRouter API](https://openrouter.ai/docs)
- 🤗 [HuggingFace TRL](https://huggingface.co/docs/trl/)
- 🔧 [PEFT/LoRA](https://huggingface.co/docs/peft/)
- 💡 [Fine-tuning Best Practices](https://huggingface.co/blog/llama2)

---

### 💡 İpuçları:

1. **Maliyet Yönetimi**: OpenRouter kullanırken token kullanımını takip edin
2. **Batch İşleme**: Büyük veri setleri için batch_size'ı optimize edin
3. **Hata Yönetimi**: max_retries parametresini ayarlayın
4. **Veri Çeşitliliği**: Farklı temperature değerleri ile deneyler yapın
5. **Kalite Kontrolü**: Üretilen verileri manuel olarak inceleyin

---

## 8. Dosya Özeti ve Temizlik

In [ ]:
# Tüm üretilen dosyaları listele ve özetle
print("\n" + "="*80)
print("📂 Üretilen Dosyalar")
print("="*80 + "\n")

total_size = 0
file_list = []

for file_type in ['*.jsonl', '*.yaml', '*.py', '*.png']:
    files = sorted(output_dir.glob(file_type)) + sorted(config_dir.glob(file_type))
    for file in files:
        if file.exists():
            size = file.stat().st_size
            size_mb = size / (1024 * 1024)
            total_size += size
            
            # Dosya türüne göre emoji
            emoji = {
                '.jsonl': '📊',
                '.yaml': '⚙️',
                '.py': '🐍',
                '.png': '🖼️'
            }.get(file.suffix, '📄')
            
            file_list.append((file, size_mb, emoji))

# Boyuta göre sırala (büyükten küçüğe)
file_list.sort(key=lambda x: x[1], reverse=True)

for file, size_mb, emoji in file_list:
    print(f"{emoji} {file.name:50s} - {size_mb:8.2f} MB")

print("\n" + "="*80)
print(f"📦 Toplam {len(file_list)} dosya - {total_size / (1024 * 1024):.2f} MB")
print("="*80)

# Google Drive'a kaydetme talimatları
print("\n💾 Google Drive'a Kaydetme:")
print("-" * 80)
print("""
# Sol panelde 'Files' ikonuna tıklayın
# 'synthetic_datasets' klasörüne sağ tıklayın
# 'Download' seçeneğini seçin
#
# Veya Google Drive mount edin:

from google.colab import drive
drive.mount('/content/drive')

# Dosyaları kopyalayın:
!cp -r synthetic_datasets /content/drive/MyDrive/
!cp -r deepfabric_configs /content/drive/MyDrive/
""")

---

## 🎉 Tebrikler!

DeepFabric ile OpenRouter GPT-OSS-20B modeli kullanarak başarıyla sentetik veri ürettiniz!

Bu notebook'u geliştirmek için önerileriniz varsa lütfen paylaşın.

---

**Hazırlayan**: DeepFabric + OpenRouter Integration  
**Tarih**: 2025  
**Versiyon**: 1.0  

---